# Sampling

The satellite imagery data used in this project was obtained from Sentinel-2 using Google Earth Engine (GEE). 
You can find the code used for data extraction here: [GEE Sentinel-2 Extraction Code](https://code.earthengine.google.com/c11abab413d190dc51fb51bb76540713) and [GEE Sentienl-2 Augmentation code](https://code.earthengine.google.com/4b8c43c9e7234ece1257f80c8d7cf56b?hl=fr)

After downloading, the imagery was divided into 256x256 pixel tiles using QGIS to facilitate processing and annotation.

In the following sections, we will continue with the labelling process and prepare the datasets for training, validation, and testing.


## Setup 

In [8]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
import numpy as np
import os
import glob
from skimage.feature import graycomatrix, graycoprops
import geopandas as gpd
from rasterio.features import rasterize
from shapely.geometry import box
from pathlib import Path

## Overview

In [127]:
# County, Prefecture, Tiff, Tiles data
data = [
    {"County": "Shuangfeng", "Prefecture": "Loudi", "Tiff": 4, "Tiles": 725},
    {"County": "Baihe", "Prefecture": "Ankang", "Tiff": 2, "Tiles": 475},
    {"County": "Hanbin", "Prefecture": "Ankang", "Tiff": 9, "Tiles": 1872},
    {"County": "Yiyuan", "Prefecture": "Zibo", "Tiff": 4, "Tiles": 806},
    {"County": "Cao", "Prefecture": "Heze", "Tiff": 4, "Tiles": 891},
    {"County": "Yuechi", "Prefecture": "Guang'an", "Tiff": 4, "Tiles": 729},
    {"County": "Jingyang", "Prefecture": "LiangshanYi", "Tiff": 4, "Tiles": 725},
    {"County": "LanpingBaiandPumi", "Prefecture": "NujiangLisu", "Tiff": 6, "Tiles": 1440},
    {"County": "Yongshan", "Prefecture": "Zhaotong", "Tiff": 6, "Tiles": 1700},
    {"County": "Yanjin", "Prefecture": "Xinxiang", "Tiff": 4, "Tiles": 420},
    {"County": "Yuanling", "Prefecture": "Huaihua", "Tiff": 9, "Tiles": 2068},
    {"County": "Sheqi", "Prefecture": "Nanyang", "Tiff": 2, "Tiles": 380},
    {"County": "Gushi", "Prefecture": "Xinyang", "Tiff": 6, "Tiles": 1118},
    {"County": "Xuanhua", "Prefecture": "Zhangjiakou", "Tiff": 4, "Tiles": 1368},
    {"County": "Daming", "Prefecture": "Handan", "Tiff": 4, "Tiles": 529},
    {"County": "Heshui", "Prefecture": "Qingyang", "Tiff": 6, "Tiles": 1672},
]

df = pd.DataFrame(data)

In [128]:
print(df['Tiles'].sum())

16918


In [129]:
total_sample_tiles_1000 = 1000 
total_sample_tiles_2000 = 2000
total_sample_tiles_5000 = 5000
total_sample_tiles_10000 = 10000

# Create a new DataFrame for proportional sampling
df_proportion = df.copy()
df_proportion['Proportion'] = df_proportion['Tiles'] / df_proportion['Tiles'].sum()
df_proportion['SampledTiles_1000'] = (df_proportion['Proportion'] * total_sample_tiles_1000).round().astype(int)
df_proportion['SampledTiles_2000'] = (df_proportion['Proportion'] * total_sample_tiles_2000).round().astype(int)
df_proportion['SampledTiles_5000'] = (df_proportion['Proportion'] * total_sample_tiles_5000).round().astype(int)
df_proportion['SampledTiles_10000'] = (df_proportion['Proportion'] * total_sample_tiles_10000).round().astype(int)

print(df_proportion[['County', 'Tiles', 'Proportion', 'SampledTiles_1000','SampledTiles_2000','SampledTiles_5000','SampledTiles_10000']])


               County  Tiles  Proportion  SampledTiles_1000  \
0          Shuangfeng    725    0.042854                 43   
1               Baihe    475    0.028077                 28   
2              Hanbin   1872    0.110651                111   
3              Yiyuan    806    0.047642                 48   
4                 Cao    891    0.052666                 53   
5              Yuechi    729    0.043090                 43   
6            Jingyang    725    0.042854                 43   
7   LanpingBaiandPumi   1440    0.085116                 85   
8            Yongshan   1700    0.100485                100   
9              Yanjin    420    0.024826                 25   
10           Yuanling   2068    0.122237                122   
11              Sheqi    380    0.022461                 22   
12              Gushi   1118    0.066083                 66   
13            Xuanhua   1368    0.080861                 81   
14             Daming    529    0.031268               

## Find positive Sample

In [130]:
focus_counties = pd.read_csv('../../focus_counties.csv')

focus_counties

,province,prefecture,county,geometry
0,Hebei,Handan,Daming,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
1,Hebei,Zhangjiakou,Xuanhua,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
2,Henan,Xinyang,Gushi,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
3,Henan,Nanyang,Sheqi,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
4,Henan,Xinxiang,Yanjin,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
5,Hunan,Huaihua,Yuanling,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
6,Hunan,Loudi,Shuangfeng,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
7,Shandong,Zibo,Yiyuan,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
8,Shandong,Heze,Cao,"{'type': 'MultiPolygon', 'coordinates': [(((11..."
9,Shaanxi,Ankang,Baihe,"{'type': 'MultiPolygon', 'coordinates': [(((11..."


In [131]:
chinapv_vectorized_2020 = gpd.read_file('../../data/data_processed/chinapv_vectorized_2020.geojson')


In [132]:
# Adjust county names for special cases before matching
chinapv_vectorized_2020_adj = chinapv_vectorized_2020.copy()
chinapv_vectorized_2020_adj['County'] = chinapv_vectorized_2020_adj.apply(
    lambda row: 'Ankang' if row['County'].lower() == 'hanbin' else (
        'Qingyang' if row['County'].lower() == 'heshui' else row['County']
    ),
    axis=1
)

positive_samples= chinapv_vectorized_2020_adj[
    chinapv_vectorized_2020_adj[['Province', 'Prefecture', 'County']].apply(
        lambda row: ((row['Province'], row['Prefecture'], row['County'])) in 
        list(zip(focus_counties['province'], focus_counties['prefecture'], focus_counties['county'])), axis=1)
]

In [58]:
positive_samples.head()

,Latitude,Longitude,Province,Area,Perimeter,urban,Prefecture,County,geometry
112,26.4986,99.4311,Yunnan,0.264018,4.715096,0,NujiangLisu,LanpingBaiandPumi,"POLYGON Z ((99.431 26.502 0, 99.432 26.502 0, ..."
432,31.9221,115.9240,Henan,0.002772,0.321835,0,Xinyang,Gushi,"POLYGON Z ((115.92 31.915 0, 115.92 31.915 0, ..."
475,32.2910,115.8070,Henan,0.016298,0.554750,0,Xinyang,Gushi,"POLYGON Z ((115.81 32.29 0, 115.81 32.29 0, 11..."
578,31.8983,115.8740,Henan,0.009142,0.382482,1,Xinyang,Gushi,"POLYGON Z ((115.87 31.899 0, 115.87 31.898 0, ..."
582,31.9766,115.5860,Henan,0.009134,0.399608,0,Xinyang,Gushi,"POLYGON Z ((115.59 31.976 0, 115.59 31.976 0, ..."


In [133]:
positive_samples.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 126 entries, 112 to 10686
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Latitude    126 non-null    float64 
 1   Longitude   126 non-null    float64 
 2   Province    126 non-null    object  
 3   Area        126 non-null    float64 
 4   Perimeter   126 non-null    float64 
 5   urban       126 non-null    int32   
 6   Prefecture  126 non-null    object  
 7   County      126 non-null    object  
 8   geometry    126 non-null    geometry
dtypes: float64(4), geometry(1), int32(1), object(3)
memory usage: 9.4+ KB


In [134]:
positive_samples_augmented= chinapv_vectorized_2020[
    chinapv_vectorized_2020[['Province', 'Prefecture']].apply(
        lambda row: ((row['Province'], row['Prefecture'])) in 
        list(zip(focus_counties['province'], focus_counties['prefecture'])), axis=1)
]

In [135]:
positive_samples_augmented.head()

,Latitude,Longitude,Province,Area,Perimeter,urban,Prefecture,County,geometry
41,41.5087,114.0300,Hebei,5.082834,16.732663,0,Zhangjiakou,Shangyi,"POLYGON Z ((114.02 41.52 0, 114.02 41.519 0, 1..."
111,41.4512,113.9540,Hebei,0.561444,4.396728,0,Zhangjiakou,Shangyi,"POLYGON Z ((113.95 41.456 0, 113.95 41.455 0, ..."
112,26.4986,99.4311,Yunnan,0.264018,4.715096,0,NujiangLisu,LanpingBaiandPumi,"POLYGON Z ((99.431 26.502 0, 99.432 26.502 0, ..."
124,41.4491,113.9690,Hebei,0.496769,3.611616,1,Zhangjiakou,Shangyi,"POLYGON Z ((113.97 41.453 0, 113.97 41.452 0, ..."
131,41.5727,114.9820,Hebei,0.362617,2.987831,1,Zhangjiakou,Guyuan,"POLYGON Z ((114.98 41.575 0, 114.98 41.575 0, ..."


In [136]:
positive_samples_augmented.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 680 entries, 41 to 10733
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Latitude    680 non-null    float64 
 1   Longitude   680 non-null    float64 
 2   Province    680 non-null    object  
 3   Area        680 non-null    float64 
 4   Perimeter   680 non-null    float64 
 5   urban       680 non-null    int32   
 6   Prefecture  680 non-null    object  
 7   County      680 non-null    object  
 8   geometry    680 non-null    geometry
dtypes: float64(4), geometry(1), int32(1), object(3)
memory usage: 50.5+ KB


In [137]:
# Group by "Prefecture" and count the number of solar panel samples per prefecture

# Count the number of samples per prefecture in each DataFrame
nb_prefecture = positive_samples.groupby("Prefecture")['geometry'].count().rename('nb_positive')
nb_prefecture_augmented = positive_samples_augmented.groupby("Prefecture")['geometry'].count().rename('nb_augmented')

# Combine the counts into a single DataFrame
comparaison_augmented = pd.concat([nb_prefecture, nb_prefecture_augmented], axis=1)

# Replace NaN values with 0
comparaison_augmented = comparaison_augmented.fillna(0)

# Calculate the difference
comparaison_augmented['difference'] = comparaison_augmented['nb_augmented'] - comparaison_augmented['nb_positive']

comparaison_augmented

,nb_positive,nb_augmented,difference
Prefecture,,,
Handan,7.0,47,40.0
Heze,15.0,46,31.0
NujiangLisu,6.0,6,0.0
Xinxiang,2.0,20,18.0
Xinyang,5.0,15,10.0
Zhangjiakou,80.0,388,308.0
Zibo,11.0,36,25.0
Ankang,0.0,1,1.0
Huaihua,0.0,2,2.0


In [138]:
comparaison_augmented.sum()

nb_positive     126.0
nb_augmented    680.0
difference      554.0
dtype: float64

In [139]:
# Ensure 'Tiles' column is added correctly by aligning index
comparaison_augmented['Tiles'] = df.groupby("Prefecture")['Tiles'].sum()

# Calculate the proportion of tiles per prefecture
comparaison_augmented['Proportion'] = comparaison_augmented['Tiles'] / comparaison_augmented['Tiles'].sum()

# Select only the 'Proportion' and 'nb_augmented' columns for display
comparaison_augmented = comparaison_augmented[['Proportion', 'nb_augmented']]

total_sample = comparaison_augmented['nb_augmented'].sum()
comparaison_augmented['sample_proportional'] =  (comparaison_augmented['Proportion'] * total_sample).round().astype(int)

comparaison_augmented

C:\Users\charl\AppData\Local\Temp\ipykernel_5444\2743737522.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparaison_augmented['sample_proportional'] =  (comparaison_augmented['Proportion'] * total_sample).round().astype(int)


,Proportion,nb_augmented,sample_proportional
Prefecture,,,
Handan,0.036510,47,25
Heze,0.061495,46,42
NujiangLisu,0.099386,6,68
Xinxiang,0.028988,20,20
Xinyang,0.077162,15,52
Zhangjiakou,0.094416,388,64
Zibo,0.055628,36,38
Ankang,0.161985,1,110
Huaihua,0.142729,2,97


The difference between the number of samples and the number of proportional samples is sometimes quite large.
For example, some prefectures like Zhangjiakou are oversampled, while others like Ankang are poorly sampled.
To address this imbalance:
- For oversampled prefectures such as Zhangjiakou, we can keep only the urban samples, since our goal is to improve performance on residential/rooftop solar panels.
- For under-sampled prefectures, we can apply data augmentation or transformations to increase their sample size.


In [180]:
# For prefecture 'Zhangjiakou', keep only urban=1 for counties that are not the focus county
focus_county = focus_counties.loc[focus_counties['prefecture'] == 'Zhangjiakou', 'county'].iloc[0]

print('Focus county for Zhangjiakou:', focus_county)

# Use positive_samples_augmented for filtering, not positive_samples
mask = ~(
    (positive_samples_augmented['Prefecture'] == 'Zhangjiakou') &
    (positive_samples_augmented['County'] != focus_county) &
    (positive_samples_augmented['urban'] != 1)
)
positive_samples_correction = positive_samples_augmented[mask]

Focus county for Zhangjiakou: Xuanhua


In [185]:
positive_samples_correction

,Latitude,Longitude,Province,Area,Perimeter,urban,Prefecture,County,geometry
112,26.4986,99.4311,Yunnan,0.264018,4.715096,0,NujiangLisu,LanpingBaiandPumi,"POLYGON Z ((99.431 26.502 0, 99.432 26.502 0, ..."
124,41.4491,113.9690,Hebei,0.496769,3.611616,1,Zhangjiakou,Shangyi,"POLYGON Z ((113.97 41.453 0, 113.97 41.452 0, ..."
131,41.5727,114.9820,Hebei,0.362617,2.987831,1,Zhangjiakou,Guyuan,"POLYGON Z ((114.98 41.575 0, 114.98 41.575 0, ..."
135,28.1917,102.5010,Sichuan,0.391145,5.752748,0,LiangshanYi,Xide,"POLYGON Z ((102.5 28.196 0, 102.5 28.194 0, 10..."
153,36.6606,113.6430,Hebei,0.023683,0.670427,0,Handan,She,"POLYGON Z ((113.64 36.66 0, 113.64 36.66 0, 11..."
...,...,...,...,...,...,...,...,...,...
10686,36.0884,117.9870,Shandong,0.024399,0.801757,0,Zibo,Yiyuan,"POLYGON Z ((117.99 36.087 0, 117.99 36.087 0, ..."
10697,36.6614,118.0890,Shandong,0.065085,1.103224,1,Zibo,Zichuan,"POLYGON Z ((118.09 36.662 0, 118.09 36.661 0, ..."
10698,36.6656,118.0900,Shandong,0.068542,1.104506,1,Zibo,Zichuan,"POLYGON Z ((118.09 36.666 0, 118.09 36.665 0, ..."
10732,26.1767,102.0530,Sichuan,0.012066,0.444242,0,LiangshanYi,Huili,"POLYGON Z ((102.05 26.177 0, 102.05 26.176 0, ..."


In [182]:
nb_prefecture = positive_samples.groupby("Prefecture")['geometry'].count().rename('nb_positive')
nb_prefecture_augmented = positive_samples_augmented.groupby("Prefecture")['geometry'].count().rename('nb_augmented')
nb_prefecture_augmented_correction = positive_samples_correction.groupby("Prefecture")['geometry'].count().rename('nb_augmented_correction')

# Combine the counts into a single DataFrame
comparaison_correction = pd.concat([nb_prefecture, nb_prefecture_augmented, nb_prefecture_augmented_correction], axis=1)

# Replace NaN values with 0
comparaison_correction = comparaison_correction.fillna(0)

# Calculate the difference
comparaison_correction['difference'] = comparaison_correction['nb_augmented_correction'] - comparaison_correction['nb_positive']

comparaison_correction

,nb_positive,nb_augmented,nb_augmented_correction,difference
Prefecture,,,,
Handan,7.0,47,47,40.0
Heze,15.0,46,46,31.0
NujiangLisu,6.0,6,6,0.0
Xinxiang,2.0,20,20,18.0
Xinyang,5.0,15,15,10.0
Zhangjiakou,80.0,388,103,23.0
Zibo,11.0,36,36,25.0
Ankang,0.0,1,1,1.0
Huaihua,0.0,2,2,2.0


In [183]:
comparaison_correction.sum()

nb_positive                126.0
nb_augmented               680.0
nb_augmented_correction    395.0
difference                 269.0
dtype: float64

In [184]:
# Ensure 'Tiles' column is added correctly by aligning index
comparaison_correction['Tiles'] = df.groupby("Prefecture")['Tiles'].sum()

# Calculate the proportion of tiles per prefecture
comparaison_correction['Proportion'] = comparaison_correction['Tiles'] / comparaison_correction['Tiles'].sum()

# Select only the 'Proportion' and 'nb_augmented_correction' columns for display
comparaison_correction = comparaison_correction[['Proportion', 'nb_augmented_correction']]

total_sample = comparaison_correction['nb_augmented_correction'].sum()
comparaison_correction['sample_proportional'] = (comparaison_correction['Proportion'] * total_sample).round().astype(int)

comparaison_correction


C:\Users\charl\AppData\Local\Temp\ipykernel_5444\2448089649.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparaison_correction['sample_proportional'] = (comparaison_correction['Proportion'] * total_sample).round().astype(int)


,Proportion,nb_augmented_correction,sample_proportional
Prefecture,,,
Handan,0.036510,47,14
Heze,0.061495,46,24
NujiangLisu,0.099386,6,39
Xinxiang,0.028988,20,11
Xinyang,0.077162,15,30
Zhangjiakou,0.094416,103,37
Zibo,0.055628,36,22
Ankang,0.161985,1,64
Huaihua,0.142729,2,56


The sample distribution is still unbalanced across prefectures, but it is improved compared to before.
Applying further transformations such as data augmentation will also help to balance the dataset.

In [194]:
positive_samples_correction.to_csv('../../data/positive_samples.csv', index=False, encoding="utf-8")

## Calculate additional indices

At this stage, I have downloaded my satellite image data from Google Earth Engine (GEE) and used QGIS to cut the large images into smaller tiles for further analysis. 

The next step is to calculate additional spectral indices for each tile, such as NDVI, NDBI, mNDWI, and NSPI, which are commonly used in remote sensing to extract information about vegetation, built-up areas, water bodies and soil properties.

Here is what each band represents:
- B1 to B7: These are the original spectral bands from the satellite imagery (e.g., Landsat or Sentinel), each capturing reflectance in different parts of the electromagnetic spectrum.
- NDVI (Normalized Difference Vegetation Index): Highlights vegetation by comparing the near-infrared (NIR) and red bands. High NDVI values indicate healthy vegetation.
- NDBI (Normalized Difference Built-up Index): Used to identify built-up (urban) areas by comparing the shortwave infrared (SWIR) and NIR bands.
- mNDWI (Modified Normalized Difference Water Index): Enhances open water features by using green and SWIR bands.
- NSPI_1 and NSPI_2 (Normalized Soil and Plant Indices): These are custom indices designed to highlight specific soil or plant characteristics, depending on the band combinations used.

Calculating these indices allows for more effective classification and analysis of land cover types in each tile. 

Note: All of these indices could have been calculated directly within GEE before downloading the data, which would have streamlined the workflow. However, in this case, I am performing the calculations locally after tiling in QGIS.


In [9]:
# Function for normalized difference index
def norm_diff(band1, band2):
    bottom = band1 + band2
    bottom[bottom == 0] = 1e-10  # avoid division by zero
    return (band1 - band2) / bottom

def quantize(arr, levels=64):
    arr_min = np.nanmin(arr)
    arr_max = np.nanmax(arr)
    scaled = ((arr - arr_min) / (arr_max - arr_min) * (levels - 1)).astype(np.uint8)
    return scaled

def calc_sum_average(arr):
    # Compute GLCM (grey level co-occurrence matrix)
    glcm = graycomatrix(arr, distances=[1], angles=[0], levels=64, symmetric=True, normed=True)
    # sum average is sum over (i+j)*p(i,j)
    i, j = np.ogrid[0:64, 0:64]
    sum_avg = np.sum((i + j) * glcm[:, :, 0, 0])
    return sum_avg

# Path to the root directory containing all folders (e.g., Baihe, Cao Augmented, Daming, etc.)
root_dir = r"D:\solar-data-china\tiles"

# Loop through each folder in the root directory
for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue
    print(f"Processing folder: {folder_name}")

    # Find all .tif files in this folder
    tiff_files = glob.glob(os.path.join(folder_path, "*.tif"))
    print(f" Found {len(tiff_files)} tiff files.")

    files_processed = 0
    files_skipped = 0
    files_error = 0

    for tiff_path in tiff_files:
        tiff_name = os.path.basename(tiff_path)
        try:
            with rasterio.open(tiff_path) as src:
                bands = src.read()  # shape = (n_bands, height, width)
                profile = src.profile

            # Check if there are at least 11 bands
            if bands.shape[0] < 11:
                # print(f"Skipping {tiff_name}: not enough bands ({bands.shape[0]})")
                files_skipped += 1
                continue

            # Extract needed bands (adjust indices accordingly)
            B1 = bands[0]
            B2 = bands[1]
            B3 = bands[2]
            B4 = bands[3]
            B5 = bands[4]
            B6 = bands[5]
            B7 = bands[6]
            B8 = bands[7]
            B9 = bands[8]
            B11 = bands[9]
            B12 = bands[10]

            # Calculate indices
            NDVI = norm_diff(B8, B4)
            NDBI = norm_diff(B11, B8)
            mNDWI = norm_diff(B3, B11)
            NSPI_1 = norm_diff(B11, B12)
            NSPI_2 = norm_diff(B11, B9)

            # Prepare list of bands and indices for texture calculation
            textures_input = {
                'B2': B2,
                'B6': B6,
                'B7': B7,
                'NDVI': NDVI,
                'mNDWI': mNDWI,
                'NDBI': NDBI
            }

            textures = {}
            for name, arr in textures_input.items():
                arr_q = quantize(arr)
                sum_avg = calc_sum_average(arr_q)
                textures[name + '_sumAvg'] = sum_avg

            # Create new stack with original bands + indices 
            new_bands = np.stack([B1, B2, B3, B4, B5, B6, B7, NDVI, NDBI, mNDWI, NSPI_1, NSPI_2])

            profile.update(count=len(new_bands), dtype=rasterio.float32)

            # Output file path
            out_name = tiff_name
            output_folder = folder_name + '_with_indices'
            output_folder_path = os.path.join(root_dir, output_folder)
            os.makedirs(output_folder_path, exist_ok=True)
            out_path = os.path.join(output_folder_path, out_name)

            with rasterio.open(out_path, 'w', **profile) as dst:
                for i, band in enumerate(new_bands, start=1):
                    dst.write(band.astype(rasterio.float32), i)

            files_processed += 1

        except Exception as e:
            # print(f"Error processing {tiff_name}: {e}")
            files_error += 1

    print(f"Finished folder: {folder_name} | Processed: {files_processed} | Skipped: {files_skipped} | Errors: {files_error}")

print("Processing complete.")


Processing folder: Augmented
 Found 286 tiff files.
Saved: s2_Handan_augmentation_0_1_1.tif
Saved: s2_Handan_augmentation_10_1_1.tif
Saved: s2_Handan_augmentation_11_1_1.tif
Saved: s2_Handan_augmentation_11_1_2.tif
Saved: s2_Handan_augmentation_12_1_1.tif
Saved: s2_Handan_augmentation_13_1_1.tif
Saved: s2_Handan_augmentation_14_1_1.tif


C:\Users\charl\AppData\Local\Temp\ipykernel_17688\240427445.py:10: RuntimeWarning: invalid value encountered in cast
  scaled = ((arr - arr_min) / (arr_max - arr_min) * (levels - 1)).astype(np.uint8)
C:\Users\charl\AppData\Local\Temp\ipykernel_17688\240427445.py:8: RuntimeWarning: All-NaN slice encountered
  arr_min = np.nanmin(arr)
C:\Users\charl\AppData\Local\Temp\ipykernel_17688\240427445.py:9: RuntimeWarning: All-NaN slice encountered
  arr_max = np.nanmax(arr)


Saved: s2_Handan_augmentation_15_1_1.tif
Saved: s2_Handan_augmentation_15_1_2.tif
Saved: s2_Handan_augmentation_16_1_1.tif
Saved: s2_Handan_augmentation_17_1_1.tif
Saved: s2_Handan_augmentation_18_1_1.tif
Saved: s2_Handan_augmentation_19_1_1.tif
Saved: s2_Handan_augmentation_1_1_1.tif
Saved: s2_Handan_augmentation_1_1_2.tif
Saved: s2_Handan_augmentation_20_1_1.tif
Saved: s2_Handan_augmentation_21_1_1.tif
Saved: s2_Handan_augmentation_22_1_1.tif
Saved: s2_Handan_augmentation_23_1_1.tif
Saved: s2_Handan_augmentation_24_1_1.tif
Saved: s2_Handan_augmentation_25_1_1.tif
Saved: s2_Handan_augmentation_26_1_1.tif
Saved: s2_Handan_augmentation_27_1_1.tif
Saved: s2_Handan_augmentation_28_1_1.tif
Saved: s2_Handan_augmentation_29_1_1.tif
Saved: s2_Handan_augmentation_2_1_1.tif
Saved: s2_Handan_augmentation_2_2_1.tif
Saved: s2_Handan_augmentation_30_1_1.tif
Saved: s2_Handan_augmentation_31_1_1.tif
Saved: s2_Handan_augmentation_32_1_1.tif
Saved: s2_Handan_augmentation_33_1_1.tif
Saved: s2_Handan_aug

C:\Users\charl\AppData\Local\Temp\ipykernel_17688\240427445.py:10: RuntimeWarning: invalid value encountered in divide
  scaled = ((arr - arr_min) / (arr_max - arr_min) * (levels - 1)).astype(np.uint8)


Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_02.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_03.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_04.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_05.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_06.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_07.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_08.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_09.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_10.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_11.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_12.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_13.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_14.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_15.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_16.tif
Saved: s2_LanpingBaiandPumi-0000004864-0000000000_12_17.tif
Saved: s2_LanpingBaiandPumi-0000004864-0

## Annotation

### Pv label

In [ ]:
# Read positive samples and set geometry
solar = gpd.read_file('../../data/positive_samples.csv')
solar = solar.set_geometry('geometry')
solar.crs = "EPSG:4326"

root_dir = r"D:\solar-data-china\tiles"

# Loop through each folder in the root directory
for folder_name in os.listdir(root_dir):
    if not folder_name.endswith("_with_indices"):
        continue

    folder_path = os.path.join(root_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue
    print(f"Processing folder: {folder_name}")

    tiles = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    for tile_path in tiles:
        with rasterio.open(tile_path) as src:
            tile_crs = src.crs
            bounds = src.bounds
            shape = (src.height, src.width)
            transform = src.transform

            # reproject solar panels to tile CRS & clip to tile extent
            solar_tile = solar.to_crs(tile_crs)
            tile_geom = box(*bounds)
            solar_clipped = solar_tile[solar_tile.intersects(tile_geom)]

            if len(solar_clipped) == 0:
                # no solar panels → mask of zeros
                mask = np.zeros(shape, dtype=np.uint8)
            else:
                # crop geometries to tile & rasterize
                mask = rasterize(
                    [(geom, 1) for geom in solar_clipped.geometry],
                    out_shape=shape,
                    transform=transform,
                    fill=0,
                    dtype=np.uint8
                )

            # save mask as GeoTIFF
            mask_path = os.path.join(os.path.dirname(folder_path), f"{os.path.splitext(os.path.basename(folder_path))[0]}_mask.tif")
            with rasterio.open(
                mask_path, 'w',
                driver='GTiff',
                height=shape[0],
                width=shape[1],
                count=1,
                dtype=mask.dtype,
                crs=tile_crs,
                transform=transform
            ) as dst:
                dst.write(mask, 1)

            print(f"Mask saved: {mask_path}")


### Non Pv label

//TODO:
Augementation : now or randomly during the training ? 
separate
train -> chose the model 
validation indices
improve with small sample pv 




run since 2015
whole china 
get the power R indices 
etc ... 

compare with the state of the art 

write the paper 